In [1]:
data_var = '2023-12-18'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7973,2023-12-18,Brasil Nbb,19:00,Cerrado,Paulistano,2.80,1.40,145.5,1.85,1.85,4.5,2.01,1.72,dhaGmUgA,0.357143,0.714286,0.540541,0.540541,0.071429,0.4,0.6,NaN,NaN,396.400,272.694505,0.687928,120.542,33.891147,0.281156,167.754,119.110,215.34,114.66,0.0,0.0,0.0,0.0,0.471405,0.000000,0.109952,0.66,0.132,13.636364,0.000000,0.0,0.000000,2.74,0.548,0.729927,0.000000,0.0,0.000000
7974,2023-12-18,Brasil Nbb,20:00,Botafogo,Fortaleza B.C.,2.09,1.69,154.5,1.81,1.85,1.5,1.94,1.77,U5cOo8OM,0.478469,0.591716,0.552486,0.540541,0.070185,0.4,0.8,NaN,NaN,373.444,92.668738,0.248146,147.876,108.599086,0.734393,251.996,272.880,299.63,339.68,0.0,0.0,0.0,0.0,0.149652,0.015456,0.064802,0.26,0.052,20.961538,0.000000,0.0,0.000000,4.37,0.874,0.789474,0.000000,0.0,0.000000
7975,2023-12-18,Brasil Nbb,20:00,Mogi,Bauru,2.65,1.44,153.5,1.88,1.79,3.5,2.08,1.66,CQ0mtnnp,0.377358,0.694444,0.531915,0.558659,0.071803,0.6,0.6,NaN,NaN,225.570,85.790446,0.380327,141.036,79.385643,0.562875,265.508,173.628,478.16,120.06,1.0,0.0,0.0,0.0,0.418386,0.034681,0.158815,-3.07,-0.614,-2.687296,0.000000,0.0,0.000000,-5.00,-1.000,-0.440000,0.000000,0.0,0.000000
7976,2023-12-18,Brasil Nbb,20:00,Pinheiros,Pato,1.37,2.91,153.5,1.85,1.85,-7.5,2.09,1.66,AHELpSwT,0.729927,0.343643,0.540541,0.540541,0.073570,0.0,0.8,NaN,NaN,165.428,54.525558,0.329603,221.328,51.689505,0.233543,152.204,237.740,190.32,259.20,0.0,0.0,0.0,0.0,0.508853,0.000000,0.162163,0.40,0.080,4.625000,0.000000,0.0,0.000000,-2.66,-0.532,-3.590226,0.000000,0.0,0.000000
7977,2023-12-18,Brasil Nbb,20:00,R10 Score Vasco da Gama,Unifacisa,1.90,1.82,145.5,1.80,1.86,-1.5,2.07,1.67,jFbKnl9G,0.526316,0.549451,0.555556,0.537634,0.075766,0.0,0.4,NaN,NaN,168.784,52.705623,0.312267,146.952,87.170135,0.593188,143.228,162.132,204.80,102.00,0.0,0.0,0.0,0.0,0.030413,0.023184,0.151253,-0.79,-0.158,-5.696203,0.000000,0.0,0.000000,-3.50,-0.700,-1.171429,0.000000,0.0,0.000000
7978,2023-12-18,Eua Nba,21:00,Cleveland Cavaliers,Houston Rockets,1.54,2.73,214.5,1.83,1.95,-6.5,2.10,1.67,4CsHjrmG,0.649351,0.366300,0.546448,0.512821,0.015651,0.4,0.6,NaN,NaN,338.306,175.000486,0.517285,228.544,132.973275,0.581828,182.192,324.108,227.33,164.80,0.0,0.0,0.0,0.0,0.394125,0.044896,0.161303,1.64,0.328,1.646341,0.630476,0.7,0.069524,0.59,0.118,14.661017,0.367652,0.2,-0.167652
7979,2023-12-18,Eua Nba,21:00,Philadelphia 76ers,Chicago Bulls,1.24,4.76,228.5,1.86,2.06,-11.5,2.07,1.85,0vyQlMHS,0.806452,0.210084,0.537634,0.485437,0.016536,0.6,1.0,NaN,NaN,159.406,15.949234,0.100054,402.576,206.939736,0.514039,166.916,480.300,135.16,324.80,0.0,0.0,0.0,0.0,0.829672,0.072154,0.079369,0.07,0.014,17.142857,0.700902,0.7,-0.000902,-0.33,-0.066,-56.969697,0.315315,0.2,-0.115315
7980,2023-12-18,Eua Nba,21:30,Miami Heat,Minnesota Timberwolves,2.14,1.83,218.5,1.83,1.91,1.5,1.97,1.85,WMReq0uj,0.467290,0.546448,0.546448,0.523560,0.013738,0.6,0.8,NaN,NaN,178.052,17.553482,0.098586,181.274,45.955297,0.253513,181.052,190.396,179.36,246.10,0.0,0.0,0.0,0.0,0.110430,0.030251,0.044426,-2.16,-0.432,-2.638889,0.607558,0.5,-0.107558,0.76,0.152,5.460526,0.587417,0.7,0.112583
7981,2023-12-18,Eua Nba,22:00,Oklahoma City Thunder,Memphis Grizzlies,1.26,4.46,225.5,1.80,2.09,-9.5,1.99,1.95,ADQarKfd,0.793651,0.224215,0.555556,0.478469,0.017866,0.8,0.6,NaN,NaN,234.262,76.163232,0.325120,277.906,52.507971,0.188941,190.120,369.458,152.76,348.40,0.0,0.0,0.0,0.0,0.791168,0.105430,0.014357,0.59,0.118,2.203390,0.610877,0.7,0.08912

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
6,21:00,Eua Nba,Philadelphia 76ers,Chicago Bulls,228.5,1.86,1.0000,Over
8,22:00,Eua Nba,Oklahoma City Thunder,Memphis Grizzlies,225.5,1.80,1.0000,Over
9,23:00,Eua Nba,Denver Nuggets,Dallas Mavericks,236.5,1.80,1.0000,Over
10,23:00,Eua Nba,Utah Jazz,Brooklyn Nets,229.5,1.80,0.9999,Over
